<a href="https://colab.research.google.com/github/Smonasteriog/HeramientasDeAnalisisV2/blob/main/Trabajo_2_An%C3%A1lisis_de_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentiment Analysis with BERT (Transfer learning, without fine-tune)

Integrantes: Francisca Gajardo - Stefany Monasterio


#Step 0: Cargar librerias

In [ ]:
import pandas as pd
import numpy as np
import random
import re

from sklearn.model_selection import train_test_split
from sklearn.utils import resample

# metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# plots
import seaborn as sns
import matplotlib.pyplot as plt



In [ ]:
def metrics_report(y_test, y_pred):
    # metrics 
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)

    return precision, recall, f1, acc

#Steps 1: Cargar base de datos

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install spacy --quiet
!python -m spacy download es_core_news_md --quiet
!python -m spacy download es_core_news_md --quiet
!python -m spacy download es_core_news_sm --quiet

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%cd '/content/gdrive/MyDrive'

In [ ]:
import datetime
import os
import pandas as pd
# Carga librerias
import warnings
warnings.filterwarnings('ignore')
import spacy
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import spacy
import os
import matplotlib.pyplot as plt
import seaborn as sns
import math
remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)
from itertools import compress
from nltk import word_tokenize
import re
from IPython.display import Image

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
esp_stop = stopwords.words('spanish')
nlp = spacy.load("es_core_news_sm")

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%ls

In [ ]:
%mkdir 'Nueva'

In [ ]:
ls

In [ ]:
cd 'Nueva'

In [ ]:
%cd '/content/drive/MyDrive/Nueva'

In [ ]:
ls

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Nueva/Coffe.csv', sep=";")

#Steps 2: Datos

In [ ]:
df.columns

Index(['name', 'roaster', 'roast', 'loc_country', 'origin_1', 'origin_2',
       '100g_USD', 'rating', 'review_date', 'desc_1', 'Unnamed: 10',
       'Unnamed: 11'],
      dtype='object')

In [ ]:
df

,name,roaster,roast,loc_country,origin_1,origin_2,100g_USD,rating,review_date,desc_1,Unnamed: 10,Unnamed: 11
0,“Sweety” Espresso Blend,A.R.C.,Medium-Light,Hong Kong,Panama,Ethiopia,14.32,95,November 2017,"Evaluated as espresso. Sweet-toned, deeply ric...",NaN,NaN
1,Flora Blend Espresso,A.R.C.,Medium-Light,Hong Kong,Africa,Asia Pacific,9.05,94,November 2017,"Evaluated as espresso. Sweetly tart, floral-to...",NaN,NaN
2,Ethiopia Shakiso Mormora,Revel Coffee,Medium-Light,United States,Guji Zone,Southern Ethiopia,4.70,92,November 2017,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",NaN,NaN
3,Ethiopia Suke Quto,Roast House,Medium-Light,United States,Guji Zone,Oromia Region,4.19,92,November 2017,"Delicate, sweetly spice-toned. Pink peppercorn...",NaN,NaN
4,Ethiopia Gedeb Halo Beriti,Big Creek Coffee Roasters,Medium,United States,Gedeb District,Gedeo Zone,4.85,94,November 2017,"Deeply sweet, subtly pungent. Honey, pear, tan...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2086,Finca Patzibir,El Gran Cafe,Medium-Light,Guatemala,Lake Atitlán Growing Region,Guatemala,4.70,92,November 2022,"Crisply sweet, nut-toned. Almond brittle, pie ...",NaN,NaN
2087,Proyecto Aurora,El Gran Cafe,Medium-Light,Guatemala,Antigua Growing Region,Guatemala,2.94,93,November 2022,"Chocolaty, floral-framed. Cocoa nib, honeysuck...",NaN,NaN
2088,Finca El Potrero,El Gran Cafe,Medium-Light,Guatemala,Antigua Growing Region,Guatemala,2.94,93,November 2022,"High-toned, enticingly sweet. Black cherry, na...",NaN,NaN
2089,Chacayá Santiago Atitlán,El Gran Cafe,Medium-Light,Guatemala,Lake Atitlán Growing Region,Guatemala,4.70,93,November 2022,"Vibrantly sweet, subtly nuanced. Apricot, dar...",NaN,NaN


In [ ]:
df_nueva = df[['loc_country', 'rating', 'desc_1']]
df_nueva

,loc_country,rating,desc_1
0,Hong Kong,95,"Evaluated as espresso. Sweet-toned, deeply ric..."
1,Hong Kong,94,"Evaluated as espresso. Sweetly tart, floral-to..."
2,United States,92,"Crisply sweet, cocoa-toned. Lemon blossom, roa..."
3,United States,92,"Delicate, sweetly spice-toned. Pink peppercorn..."
4,United States,94,"Deeply sweet, subtly pungent. Honey, pear, tan..."
...,...,...,...
2086,Guatemala,92,"Crisply sweet, nut-toned. Almond brittle, pie ..."
2087,Guatemala,93,"Chocolaty, floral-framed. Cocoa nib, honeysuck..."
2088,Guatemala,93,"High-toned, enticingly sweet. Black cherry, na..."
2089,Guatemala,93,"Vibrantly sweet, subtly nuanced. Apricot, dar..."


In [ ]:
# create labels 
label = []
for i in df_nueva.rating:
    if i > 90:
        label.append(1) # positive 
    else:
        label.append(0) # negative
df_nueva["Label"] = label
df_nueva  

,loc_country,rating,desc_1,Label
0,Hong Kong,95,"Evaluated as espresso. Sweet-toned, deeply ric...",1
1,Hong Kong,94,"Evaluated as espresso. Sweetly tart, floral-to...",1
2,United States,92,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",1
3,United States,92,"Delicate, sweetly spice-toned. Pink peppercorn...",1
4,United States,94,"Deeply sweet, subtly pungent. Honey, pear, tan...",1
...,...,...,...,...
2086,Guatemala,92,"Crisply sweet, nut-toned. Almond brittle, pie ...",1
2087,Guatemala,93,"Chocolaty, floral-framed. Cocoa nib, honeysuck...",1
2088,Guatemala,93,"High-toned, enticingly sweet. Black cherry, na...",1
2089,Guatemala,93,"Vibrantly sweet, subtly nuanced. Apricot, dar...",1


In [ ]:
df_nueva.desc_1.iloc(0)

In [ ]:
df_nueva.Label.value_counts()

1    1967
0     124
Name: Label, dtype: int64

In [ ]:
data = df_nueva['desc_1']
y = df_nueva['Label']

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, stratify=y, random_state = 124)

print("Train data:",  X_train.shape, y_train.shape)
print("Test data:",  X_test.shape, y_test.shape)

Train data: (1463,) (1463,)
Test data: (628,) (628,)


In [ ]:
# Upsampling for training
train = pd.DataFrame({'desc_1':X_train, 'Label':y_train})
#Count 1s and 0s
ones = len(train.loc[train['Label'] == 1])
zeros = len(train.loc[train['Label'] == 0])
if ones >= zeros:
    majority = 1
    minority = 0
else:
    majority = 0
    minority = 1
    
# Upsample TrainingSet
train_majority = train[train.Label==majority]
train_minority = train[train.Label==minority]

# Upsample minority class
train_minority_upsampled = resample(train_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(train_majority),    # to match majority class
                                 random_state=42) # reproducible results

# Combine majority class with upsampled minority class
train = pd.concat([train_majority, train_minority_upsampled])
X_train = train['desc_1']
y_train = train['Label']
print("Train data:",  X_train.shape, y_train.shape)
print("Test data:",  X_test.shape, y_test.shape)

Train data: (2752,) (2752,)
Test data: (628,) (628,)


#*Steps* 3: Modelo

In [ ]:
from transformers import pipeline

In [ ]:
# load the model classifier 
model_pipline = 'distilbert-base-uncased-finetuned-sst-2-english' 
classifier = pipeline('sentiment-analysis', model_pipline)

In [ ]:
classifier

In [ ]:
classifier("Good coffe")

[{'label': 'POSITIVE', 'score': 0.9998204112052917}]

In [ ]:
# take time (1 minute aprox)
max_length = 512 
Label_pred = []
rating_pred = []
for i in X_test: 
    results = classifier(i[0:max_length])
    Label_pred.append(results[0]['label'])
    rating_pred.append(results[0]['score'])

In [ ]:
# change format of labels prediction, from text to numbers
y_pred = [1 if i=='POSITIVE' else 0 for i in Label_pred]

#Steps 4: Resultados

In [ ]:
# get metrics
y = y_test.tolist()
metrics1 = metrics_report(y, y_pred)
p1 = metrics1[0]
r1 = metrics1[1]
f1 = metrics1[2]
ac1 = metrics1[3]

In [ ]:
# print results
m1 = model_pipline.split('-')[0]
d = {'Model':[m1], 'Precision':[p1], 'Recall':[r1], 'F1':[f1], 'Accuracy':[ac1]}
Bert1_results = pd.DataFrame(d)
Bert1_results

,Model,Precision,Recall,F1,Accuracy
0,distilbert,0.941083,1.0,0.969647,0.941083
